## **Example) Visualize Multi-input MNIST Activation Map (1/2)**
### *Initialize and training*

### **1. Initialize GPU memory**

In [1]:
import tensorflow as tf
# GPU constraints
config = tf.ConfigProto()
config.gpu_options.allow_growth = True 
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### **2. Load MNIST dataset**

In [2]:
import numpy as np
from keras.datasets import mnist

num_classes = 10
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(np.float32) / 255
x_train = np.expand_dims(x_train, axis=-1)
x_test = x_test.astype(np.float32) / 255
x_test = np.expand_dims(x_test, axis=-1)

print("Original MNIST dataset shapes")
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

Using TensorFlow backend.


Original MNIST dataset shapes
(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


### **3. Preprocess for multi-input classification**

In [3]:
from copy import deepcopy

def multi_input_mnist(X, Y):
    x1_indices = np.arange(len(X))
    np.random.shuffle(x1_indices)
    X1 = deepcopy(X[x1_indices])
    Y1 = deepcopy(Y[x1_indices])

    x2_indices = np.arange(len(X))
    np.random.shuffle(x2_indices)
    X2 = deepcopy(X[x2_indices])
    Y2 = deepcopy(Y[x2_indices])

    print(Y)

    Y = []
    for y1, y2 in zip(Y1, Y2):
        if y1 == y2:
            #print("y1 : {} / y2 : {}".format(y1, y2))
            Y.append([True])
        else:
            Y.append([False])

    Y = np.array(Y)

    print("Multi-input MNIST dataset shapes")
    print("x1 : {}".format(X1.shape))
    print("x2 : {}".format(X2.shape))
    print("y  : {}".format(Y.shape))
    print("number of positive : {}".format(np.sum(Y)))
    return X1, X2, Y


In [4]:
x1_train, x2_train, y_train = multi_input_mnist(x_train, y_train)

[5 0 4 ... 5 6 8]
Multi-input MNIST dataset shapes
x1 : (60000, 28, 28, 1)
x2 : (60000, 28, 28, 1)
y  : (60000, 1)
number of positive : 5955


In [5]:
x1_test, x2_test, y_test = multi_input_mnist(x_test, y_test)

[7 2 1 ... 4 5 6]
Multi-input MNIST dataset shapes
x1 : (10000, 28, 28, 1)
x2 : (10000, 28, 28, 1)
y  : (10000, 1)
number of positive : 972


In [6]:
print(y_test, np.sum(y_test), len(y_test), 1 - np.sum(y_test)/len(y_test))

[[ True]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]] 972 10000 0.9028


### **4. Initialize classification model**

In [7]:
from keras.models import Model
from keras.layers import Dense, Input, Concatenate, Conv2D, Activation, Flatten

def base_model():
    input_x1 = Input(shape=(28,28,1))
    x1 = Conv2D(filters=64, kernel_size=(8,8), padding="same", strides=1)(input_x1)
    x1 = Activation('relu')(x1)
    x1_out = x1
    x1 = Conv2D(filters=128, kernel_size=(4,4), padding="same", strides=2)(x1)
    x1 = Activation('relu')(x1)
    x1 = Conv2D(filters=128, kernel_size=(2,2), padding="same", strides=2)(x1)
    x1 = Activation('relu')(x1)
    
    input_x2 = Input(shape=(28,28,1))
    x2 = Conv2D(filters=64, kernel_size=(8,8), padding="same", strides=1)(input_x2)
    x2 = Activation('relu')(x2)
    x2_out = x2
    x2 = Conv2D(filters=128, kernel_size=(4,4), padding="same", strides=2)(x2)
    x2 = Activation('relu')(x2)
    x2 = Conv2D(filters=128, kernel_size=(2,2), padding="same", strides=2)(x2)
    x2 = Activation('relu')(x2)
    
    x1 = Flatten()(x1)
    x2 = Flatten()(x2)
    
    x = Concatenate()([x1, x2])
    x = Dense(128, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    y = x
    
    model_x1 = Model(input_x1, x1_out)
    model_x2 = Model(input_x2, x2_out)
    model = Model([input_x1, input_x2], y)
    return model, model_x1, model_x2

In [8]:
model, model_x1, model_x2 = base_model()

In [9]:
model.summary() # training model

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 64)   4160        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 28, 28, 64)   4160        input_2[0][0]                    
__________________________________________________________________________________________________
activation

In [10]:
model_x1.summary() # first layer output model of input x1

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        4160      
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 64)        0         
Total params: 4,160
Trainable params: 4,160
Non-trainable params: 0
_________________________________________________________________


In [11]:
model_x2.summary() # first layer output model of input x2

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 64)        4160      
_________________________________________________________________
activation_4 (Activation)    (None, 28, 28, 64)        0         
Total params: 4,160
Trainable params: 4,160
Non-trainable params: 0
_________________________________________________________________


### **5. Compile and train the model**

In [12]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
model.fit(x=[x1_train, x2_train], 
          y=y_train,
          batch_size=1000,
          epochs=30, 
          shuffle=True,
          validation_data=([x1_test, x2_test], y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/30
60000/60000 [==============================] - 7s 114us/step - loss: 0.3429 - acc: 0.8878 - val_loss: 0.3189 - val_acc: 0.9028
Epoch 2/30
60000/60000 [==============================] - 5s 83us/step - loss: 0.3240 - acc: 0.9007 - val_loss: 0.3186 - val_acc: 0.9028
Epoch 3/30
60000/60000 [==============================] - 5s 84us/step - loss: 0.3233 - acc: 0.9007 - val_loss: 0.3187 - val_acc: 0.9028
Epoch 4/30
60000/60000 [==============================] - 5s 84us/step - loss: 0.3234 - acc: 0.9008 - val_loss: 0.3185 - val_acc: 0.9028
Epoch 5/30
60000/60000 [==============================] - 5s 84us/step - loss: 0.3213 - acc: 0.9008 - val_loss: 0.3173 - val_acc: 0.9028
Epoch 6/30
60000/60000 [==============================] - 5s 84us/step - loss: 0.3111 - acc: 0.9007 - val_loss: 0.2760 - val_acc: 0.9028
Epoch 7/30
60000/60000 [==============================] - 5s 84us/step - loss: 0.2231 - acc: 0.9135 - val_loss: 0.1754 - val_ac

### **6. Save trained model**

In [15]:
model.save('models/example01.h5')
model_x1.save('models/example01-1.h5')
model_x2.save('models/example01-2.h5')